# Electrostatic analysis

This example shows how to use PyAEDT to create a Maxwell 2D electrostatic analysis.
It shows how to create the geometry, load material properties from a Microsoft Excel file, and
set up the mesh settings. Moreover, it focuses on postprocessing operations, in particular how to
plot field line traces, which are relevant for an electrostatic analysis.

Keywords: **Maxwell 2D**, **electrostatic**.

## Perform imports and define constants

Perform required imports.

In [1]:
import os
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_file

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``..

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download Excel file

Set the local temporary folder to export the Excel (XLSX) file to.

In [4]:
file_name_xlsx = download_file(
    source="field_line_traces", name="my_copper.xlsx", local_path=temp_folder.name
)

## Initialize dictionaries

Initialize the dictionaries that contain all the definitions for the design variables.

In [5]:
geom_params_circle = {
    "circle_x0": "-10mm",
    "circle_y0": "0mm",
    "circle_z0": "0mm",
    "circle_axis": "Z",
    "circle_radius": "1mm",
}

geom_params_rectangle = {
    "r_x0": "1mm",
    "r_y0": "5mm",
    "r_z0": "0mm",
    "r_axis": "Z",
    "r_dx": "-1mm",
    "r_dy": "-10mm",
}

## Launch AEDT and Maxwell 2D

Launch AEDT and Maxwell 2D after first setting up the project and design names,
the solver, and the version. The following code also creates an instance of the
``Maxwell2d`` class named ``m2d``.

In [6]:
project_name = os.path.join(temp_folder.name, "M2D_Electrostatic.aedt")
m2d = ansys.aedt.core.Maxwell2d(
    project=project_name,
    version=AEDT_VERSION,
    design="Design1",
    solution_type="Electrostatic",
    new_desktop=True,
    non_graphical=NG_MODE,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_433199f2-b871-468b-9fc3-a9c79f73c70a.log is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project M2D_Electrostatic has been created.


PyAEDT INFO: Added design 'Design1' of type Maxwell 2D.


PyAEDT INFO: Aedt Objects correctly read


## Set modeler units

Set modeler units to ``mm``.

In [7]:
m2d.modeler.model_units = "mm"

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


## Define variables from dictionaries

Define design variables from the created dictionaries.

In [8]:
for k, v in geom_params_circle.items():
    m2d[k] = v
for k, v in geom_params_rectangle.items():
    m2d[k] = v

## Read materials from Excel file

Read materials from the Excel file into the design.

In [9]:
mats = m2d.materials.import_materials_from_excel(file_name_xlsx)

PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


## Create design geometries

Create a rectangle and a circle. Assign the material read from the Excel file.
Create two new polylines and a region.

In [10]:
rect = m2d.modeler.create_rectangle(
    origin=["r_x0", "r_y0", "r_z0"],
    sizes=["r_dx", "r_dy", 0],
    name="Ground",
    material=mats[0],
)
rect.color = (0, 0, 255)  # rgb
rect.solve_inside = False

circle = m2d.modeler.create_circle(
    origin=["circle_x0", "circle_y0", "circle_z0"],
    radius="circle_radius",
    num_sides="0",
    is_covered=True,
    name="Electrode",
    material=mats[0],
)
circle.color = (0, 0, 255)  # rgb
circle.solve_inside = False

poly1_points = [[-9, 2, 0], [-4, 2, 0], [2, -2, 0], [8, 2, 0]]
poly2_points = [[-9, 0, 0], [9, 0, 0]]
poly1_id = m2d.modeler.create_polyline(
    points=poly1_points, segment_type="Spline", name="Poly1"
)
poly2_id = m2d.modeler.create_polyline(points=poly2_points, name="Poly2")
m2d.modeler.split(assignment=[poly1_id, poly2_id], plane="YZ", sides="NegativeOnly")
m2d.modeler.create_region(pad_value=[20, 100, 20, 100])

Region

## Define excitations

Assign voltage excitations to the rectangle and circle.

In [11]:
m2d.assign_voltage(assignment=rect.id, amplitude=0, name="Ground")
m2d.assign_voltage(assignment=circle.id, amplitude=50e6, name="50kV")

PyAEDT INFO: Boundary Voltage Ground has been created.


PyAEDT INFO: Boundary Voltage 50kV has been created.


50kV

## Create initial mesh settings

Assign a surface mesh to the rectangle.

In [12]:
m2d.mesh.assign_surface_mesh_manual(assignment=["Ground"], surface_deviation=0.001)

PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


ModelResolution_YR5YFG

## Create, validate, and analyze setup

In [13]:
setup_name = "MySetupAuto"
setup = m2d.create_setup(name=setup_name)
setup.props["PercentError"] = 0.5
setup.update()
m2d.validate_simple()
m2d.analyze_setup(name=setup_name, use_auto_settings=False, cores=NUM_CORES)

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving design setup MySetupAuto


PyAEDT INFO: Design setup MySetupAuto solved correctly in 0.0h 0.0m 18.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


True

## Evaluate the E Field tangential component

Evaluate the E Field tangential component along the given polylines.
Add these operations to the **Named Expression** list in the field calculator.

In [14]:
e_line = m2d.post.fields_calculator.add_expression(
    calculation="e_line", assignment=None
)
m2d.post.fields_calculator.expression_plot(
    calculation="e_line", assignment="Poly1", names=[e_line]
)
m2d.post.fields_calculator.expression_plot(
    calculation="e_line", assignment="Poly12", names=[e_line]
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpz1p2wgc9.ansys\M2D_Electrostatic.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpz1p2wgc9.ansys\M2D_Electrostatic.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.008372306823730469


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


[]

## Create field line traces plot

Create a field line traces plot specifying as seeding faces
the ground, the electrode, and the region
and as ``In surface objects`` only the region.

In [15]:
plot = m2d.post.create_fieldplot_line_traces(
    seeding_faces=["Ground", "Electrode", "Region"],
    in_volume_tracing_objs=["Region"],
    plot_name="LineTracesTest",
)

PyAEDT INFO: Active Design set to Design1


## Update field line traces plot

Update the field line traces plot.
Update the seeding points number, line style, and line width.

In [16]:
plot.SeedingPointsNumber = 20
plot.LineStyle = "Cylinder"
plot.LineWidth = 3
plot.update()

True

## Export field line traces plot

Export the field line traces plot.
For the field lint traces plot, the export file format is ``.fldplt``.

In [17]:
m2d.post.export_field_plot(
    plot_name="LineTracesTest", output_dir=temp_folder.name, file_format="fldplt"
)

WindowsPath('C:/Users/ansys/AppData/Local/Temp/tmpz1p2wgc9.ansys/LineTracesTest.fldplt')

## Export mesh field plot

Export the mesh to an AEDTPLT file.

In [18]:
m2d.post.export_mesh_obj(setup=m2d.nominal_adaptive)

PyAEDT INFO: Active Design set to Design1


WindowsPath('C:/Users/ansys/AppData/Local/Temp/tmpz1p2wgc9.ansys/M2D_Electrostatic.pyaedt/Design1/Mesh_1GA6DC.aedtplt')

## Release AEDT

In [19]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project M2D_Electrostatic Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [20]:
temp_folder.cleanup()